In [17]:
!pip install openai numpy 


Defaulting to user installation because normal site-packages is not writeable


In [4]:
from openai import OpenAI
import os

client = OpenAI(api_key="Hier API Key einsetzen")

query = "Wie warm ist die Jacke X-100?"
chunks = [
    "Die X-100 Jacke ist wasserdicht, windabweisend und atmungsaktiv.",
    "Sie eignet sich für Temperaturen bis -15 °C. Kritische Nähte sind verschweißt."
]

def score_chunk(chunk_text: str) -> float:
    messages = [
        {"role": "system", "content":
            "Du bist ein Assistent, der ausschließlich einen numerischen Wert zwischen 0.0 und 1.0 zurückgibt. "
            "Antworte NUR mit diesem Wert, ohne Einheit, Text oder Satzzeichen."},
        {"role": "user", "content":
            f"Frage: {query}\n"
            f"Text:  {chunk_text}\n\n"
            "Gib jetzt bitte nur die Relevanz-Zahl:"}
    ]
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0,
        max_tokens=4
    )
    text = resp.choices[0].message.content.strip()
    print(f"Raw model output for chunk: «{text}»")      # Debug
    try:
        return float(text)
    except ValueError:
        # Falls es trotzdem nicht parsebar ist, greift Regex
        import re
        m = re.search(r"0\.\d+|1\.0", text)
        return float(m.group(0)) if m else 0.0

# 4) Alle Chunks bewerten
scores = [score_chunk(c) for c in chunks]

# 5) Sortieren und ausgeben
ranked = sorted(zip(chunks, scores), key=lambda x: x[1], reverse=True)
print("Rerank-Ergebnis (Chat-Cross-Encoder):")
for i, (text, score) in enumerate(ranked, 1):
    print(f"{i}. (Score={score:.3f}) {text}")


Raw model output for chunk: «0.2»
Raw model output for chunk: «0.8»
Rerank-Ergebnis (Chat-Cross-Encoder):
1. (Score=0.800) Sie eignet sich für Temperaturen bis -15 °C. Kritische Nähte sind verschweißt.
2. (Score=0.200) Die X-100 Jacke ist wasserdicht, windabweisend und atmungsaktiv.
